# Libraries

In [2]:
import os
#os.chdir('C:/Users/Matteo Salis/Desktop/Python/water-pinns/src')  # Provide the new path here
os.chdir('/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/src')

In [3]:
from datetime import datetime

In [4]:
from operator import itemgetter
from tqdm import tqdm
import time
from datetime import datetime
import json
from collections import deque

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray
import rioxarray
import fiona

#import matplotlib
import matplotlib.pyplot as plt

#from rasterio.enums import Resampling

import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data.sampler import RandomSampler
import torch.nn as nn
from torch.autograd import Variable

import wandb

import torchview
from torchview import draw_graph

from utils.plot import *
import importlib

import seaborn as sns

In [5]:
from dataloaders import dataset_ST_MultiPoint
# from dataloaders import load_dataset
# from loss import losses_SparseData
# from models import models_SparseData
# from optimizer import load_optimizer

from subprocess import Popen

# Load dictionary

In [6]:
#json_file = "C:/Users/Matteo Salis/Desktop/Python/water-pinns/config/ST_MultiPoint/ST_MultiPoint_STDisNet_SAGW_PI_ERA5_local.json" #config_files_1d/lstm_att_1.json
json_file = "/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/config/ST_MultiPoint/ST_MultiPoint_STDisNet_SAGW_PI_ERA5.json" #config_files_1d/lstm_att_1.json
dict_files = {}
with open(json_file) as f:
    dict_files = json.load(f)
    print(f"Read data.json: {dict_files}")

Read data.json: {'cuda_device': 'cuda:0', 'seed': 42, 'entity': 'gsartor-unito', 'experiment_name': 'ST_MultiPoint', 'run_name': 'PI_S_ref', 'comments': None, 'wandb_mode': 'offline', 'wandb_dir': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/results/results_ST_MultiPoint/logs', 'save_model_dir': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/results/results_ST_MultiPoint/runs', 'dtm_nc': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/dtm_ROI.nc', 'weather_nc_path': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/era5land_2000_2024_weekly.nc', 'weather_dtm': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/era5land_dtm.nc', 'wtd_csv_path': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/dataset_wtd_roi.csv', 'wtd_shp': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/shapefile/underground_wtd_sensor_roi.shp', 'piedmont_shp': '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/shapefile/piemonte_admin_boundaries.shp', 'recharge_areas_shp': '/leonardo_

# Dataset class

In [7]:
importlib.reload(dataset_ST_MultiPoint)

<module 'dataloaders.dataset_ST_MultiPoint' from '/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/src/dataloaders/dataset_ST_MultiPoint.py'>

In [8]:
ds = dataset_ST_MultiPoint.Dataset_ST_MultiPoint(dict_files)

    Loading weather data... Done!
    Loading dtm... Done!
    Loading Piedmont Shapefiles... 

/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/src/dataloaders/dataset_ST_MultiPoint.py:101: UserWarning: Geometry is in a geographic CRS. Results from 'buffer' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  clipped_buffer = clipped.buffer(buffer_recharge)


Done!
    Loading underground water data... Done!
    Norm factors:
{'target_means': np.float64(297.1249160768408), 'target_stds': np.float64(73.38987997145817), 'dtm_mean': <xarray.DataArray '__xarray_dataarray_variable__' ()> Size: 4B
array(1139.4907, dtype=float32)
Coordinates:
    spatial_ref  int64 8B 0, 'dtm_std': <xarray.DataArray '__xarray_dataarray_variable__' ()> Size: 4B
array(884.7991, dtype=float32)
Coordinates:
    spatial_ref  int64 8B 0, 'lat_mean': np.float64(44.800499999999964), 'lat_std': np.float64(0.3169655454549103), 'lon_mean': np.float64(7.375499999999998), 'lon_std': np.float64(0.4326943495034695), 'weather_mean': <xarray.Dataset> Size: 24B
Dimensions:      ()
Coordinates:
    spatial_ref  int64 8B 0
Data variables:
    tp           float32 4B 0.001774
    t2m          float32 4B 280.5
    pev          float32 4B -0.001865
    smlt         float32 4B 0.000512, 'weather_std': <xarray.Dataset> Size: 24B
Dimensions:      ()
Coordinates:
    spatial_ref  int64 8B 0

# Maps

In [ ]:
import matplotlib
# Colormap
import cmasher as cmr
# Automatic labeling
import adjustText as aT
# define box and geometry object
import shapely
from shapely.geometry import box

In [ ]:
# Italy shp
italy_shp = gpd.read_file("/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/shapefile/Italy_Reg01012024_g_WGS84.shp", engine='fiona')
italy_shp = italy_shp.to_crs('epsg:4326')

In [9]:
# extract Province names and coord, useful for plots
provinces_names = [ds.piemonte_shp["nome"][i] for i in range(0,len(ds.piemonte_shp))]
provinces_long = [ds.piemonte_shp["geometry"][i].representative_point().coords[0][0] for i in range(0,len(ds.piemonte_shp))]
provinces_lat = [ds.piemonte_shp["geometry"][i].representative_point().coords[0][1] for i in range(0,len(ds.piemonte_shp))]
provinces_lat[-3] = provinces_lat[-3] - 0.25
provinces_long[-3] = provinces_long[-3] - 0.05

In [ ]:
# Piedmont DTM (precise border)
piemonte_dtm = rioxarray.open_rasterio("/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/DTMPiemonte_filled_50m.tif", engine='fiona')
piemonte_dtm = piemonte_dtm.rio.reproject("epsg:4326")
piemonte_dtm = piemonte_dtm.where(piemonte_dtm != -99999) # Take valid pixel

In [18]:
# Load river shp
rivers_shp_100 = gpd.read_file("/leonardo_scratch/fast/IscrC_DL4STP/water-pinns/data/shapefile/idro100-fiumi.shp", engine='fiona')
rivers_shp_100 = rivers_shp_100.to_crs('epsg:4326')

In [31]:
roi_box = [ds.dtm_roi.x.min().values,
        ds.dtm_roi.x.max().values,
        ds.dtm_roi.y.min().values,
        ds.dtm_roi.y.max().values]

In [ ]:
# Compute sensor norm factors 
target_means = ds.wtd_df[ds.target].groupby(level=1).transform('mean').values
target_means = target_means.reshape(len(ds.wtd_df.index)//len(ds.sensor_id_list),
                                    len(ds.sensor_id_list))[0,:] 
target_stds = ds.wtd_df[ds.target].groupby(level=1).transform('std').values
target_stds = target_stds.reshape(len(ds.wtd_df.index)//len(ds.sensor_id_list),
                                    len(ds.sensor_id_list))[0,:]

In [33]:
fig, ax = plt.subplots(figsize = (7,7))

# Log-Scale: Define the ticks
cnorm = matplotlib.colors.LogNorm(vmin=piemonte_dtm.min(), vmax=piemonte_dtm.max())
decades = np.ceil(np.log10(cnorm.vmax / cnorm.vmin))
formatter = matplotlib.ticker.LogFormatter(10, minor_thresholds=(decades, decades))

# Plot with lognorm transformation
piemonte_elevation = piemonte_dtm.plot(ax = ax, add_colorbar=False,
                                      cmap = cmr.get_sub_cmap(plt.cm.terrain, 0.25, 1),
                                      norm = matplotlib.colors.LogNorm(vmin = piemonte_dtm.min().values,
                                                                       vmax = piemonte_dtm.max().values))
ax.set_title("Piedmont (IT)")

# Regional shapefile
ds.piemonte_shp.boundary.plot(color = 'Black', linewidth = 0.5, alpha = 0.6, ax = ax, label = "Administrative bounds")

# Rivers
rivers_shp_100.loc[rivers_shp_100["CLASSE"] == 1].plot(ax = ax, label = "100", linewidth=0.5)


# Sensors
ds.wtd_geodf.plot(color = "yellow", ax = ax,
                        markersize = 15, edgecolors = "darkred",
                        linewidths = 0.75,
                        label="WTD sensors",
                         zorder=7)

# Provinces labels
texts = []
for x, y, label in zip(provinces_long, provinces_lat, provinces_names):

    texts.append(plt.text(x,y,label, ha='center', va='center' ,
                          fontsize =10, fontweight = "bold", fontstyle = "italic", color = "Black"))

    aT.adjust_text(texts, force_text =(0.1, 0.1), force_explode=(0, 0), #, min_arrow_len = 5
                       force_static =(0, 0),
                       arrowprops=dict(arrowstyle="simple", linestyle = "--", color='dimgrey', lw=1.2))
    




ax.plot(*roi_box.exterior.xy, label = "Meteo-box", color = "magenta")

# Inset
#inset_object = [italy_borders, Piemonte_box.exterior.xy]
piemonte_long_min, piemonte_lat_min, piemonte_long_max, piemonte_lat_max = ds.piemonte_shp.total_bounds
piemonte_box = box(piemonte_long_min, piemonte_lat_min, piemonte_long_max , piemonte_lat_max)

box(piemonte_long_min, piemonte_lat_min, piemonte_long_max , piemonte_lat_max)
axins = ax.inset_axes(bounds = [0,0.65,0.35,0.35])
italy_shp.boundary.plot(ax = axins, color = 'gray', linewidth = 0.5) # Italy
axins.plot(*piemonte_box.exterior.xy, color = "black") # Piemonte
axins.set_xticks([])
axins.set_yticks([])

# add min and max values

ticks = [cnorm.vmin, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000]
ticks.append(cnorm.vmax)
fig.colorbar(ticks=ticks, mappable=piemonte_elevation,
                  format = formatter, label = "DTM [m]")

## North arrow
north_x_shift = 1.05
north_y_shift = -0.18
north_fontsize = 15
arrow_dx = 0
arrow_dy = 0.1
arrow_x_shift = 1.105
arrow_y_shift = -0.3
arrow_head_width=0.08
arrow_head_length=0.1
ax.text(x=ax.get_xlim()[0]+north_x_shift, y=ax.get_ylim()[1]+north_y_shift, s='N', fontsize=north_fontsize)
ax.arrow(ax.get_xlim()[0]+arrow_x_shift, ax.get_ylim()[1]+arrow_y_shift, arrow_dx, arrow_dy, length_includes_head=True,
         head_width=arrow_head_width, head_length=arrow_head_length, overhang=.1, facecolor='k')

plt.legend(bbox_to_anchor=(0.825, -0.085) )

#plt.savefig(f"{saving_plots_dir}roi_piemonte.png", dpi = 400, transparent = True, bbox_inches = 'tight')

plt.show()

: 

# Time Series

In [ ]:
weekly_sensor_list = ds.sensor_id_list
for sensor_idx in range(len(weekly_sensor_list)):
        
    sensor_series = ds.wtd_df["wtd"].loc[pd.IndexSlice[:, weekly_sensor_list[sensor_idx]]]
    
    #sensor_series = (sensor_series * ds.norm_factors["target_std"]) + ds.norm_factors["target_mean"]
    
    sensor_series_dates = sensor_series.index.get_level_values(0)
    
    municipality = ds.wtd_names.loc[ds.wtd_names["sensor_id"] == weekly_sensor_list[sensor_idx],
                                    "munic"].values[0]
    
    fig, ax = plt.subplots(figsize = (25,4))
    
    fig.suptitle(f"IDW STATS {municipality} ID Sensor: {weekly_sensor_list[sensor_idx]}")
    
    ax.plot(sensor_series_dates,
            sensor_series.values, 
            marker = "o",
            lw = 0.5,
            markersize = 1.5)
    
    ax.set_ylabel("H")
    
    date_xticks = pd.date_range(np.datetime64("2001-01-01"), np.datetime64("2023-12-31"), freq = "6MS",  normalize = True, inclusive = "both")
    
    ax.set_ylim([sensor_series.min(),
                 sensor_series.max()])
    
    ax.bar(sensor_series_dates[sensor_series.isnull()],
                      bottom = ax.get_ylim()[0],
                      height = ax.get_ylim()[1],
                      width= 5,
                      align='center',
                      color = 'darkred',
                      alpha = 0.4,
                      label = "Water Table\nMissing Values", zorder = 0)
    
    ax.set_xticks(date_xticks, date_xticks.strftime('%d/%m/%Y'))
    ax.tick_params(axis = "x", rotation=50 )
    ax.grid(axis="x", ls = "--", which = "both", lw = "1.5")